In [93]:
#finding common strains
import pandas as pd

#load files
spike = pd.read_csv("spike.csv", sep=",")
nsp12 = pd.read_csv("nsp12.csv", sep=",")
nuc = pd.read_csv("nucleocapsid.csv", sep=",")

spike.count()


Accession                 68
Organism_Name             68
GenBank_RefSeq            68
Assembly                  68
Nucleotide                68
Submitters                67
Release_Date              68
Isolate                   36
Species                   68
Length                    68
Protein                   68
Geo_Location              57
USA                        3
Host                      55
Tissue_Specimen_Source     9
Collection_Date           49
dtype: int64

In [94]:
#extract organism name
spike_names = set(spike['Organism_Name']) 
nsp12_names = set(nsp12['Organism_Name'])
nuc_names = set(nuc['Organism_Name'])

#shared organism name
shared = spike_names & nsp12_names & nuc_names

print("Found", len(shared), "organism names") 


Found 15 organism names


In [95]:
shared_spike_nsp12 = spike_names & nsp12_names
print("Found", len(shared_spike_nsp12), "organism names") 

Found 16 organism names


In [96]:
shared_nsp12_nuc = nsp12_names & nuc_names
print("Found", len(shared_nsp12_nuc), "organism names")

Found 17 organism names


In [97]:
shared_nuc_spike = spike_names & nuc_names
print("Found", len(shared_nuc_spike), "organism names") 

Found 57 organism names


In [98]:
#check with assembly
spike_assembly = set(spike['Assembly']) 
nsp12_assembly = set(nsp12['Assembly']) 
nuc_assembly = set(nuc['Assembly']) 

shared_nuc_spike_assembly = spike_assembly & nuc_assembly
print("Found", len(shared_nuc_spike_assembly), "assemblies") 



Found 60 assemblies


In [99]:
print(shared_nuc_spike_assembly)


{'GCF_001500975.1', 'GCF_000886515.1', 'GCF_000896875.1', 'GCF_023141795.1', 'GCF_001503155.1', 'GCF_023148835.1', 'GCF_000879255.1', 'GCF_012271615.1', 'GCF_001661775.1', 'GCF_000895415.1', 'GCF_012271625.1', 'GCF_000873025.1', 'GCF_012271565.1', 'GCF_002194405.1', 'GCF_002118885.1', 'GCF_001962315.1', 'GCF_000856025.1', 'GCF_000887595.1', 'GCF_000848685.1', 'GCF_000853865.1', 'GCF_000926915.1', 'GCF_003971785.1', 'GCF_001505415.1', 'GCF_012271635.1', 'GCF_001501755.1', 'GCF_000930095.1', 'GCF_012271745.1', 'GCF_023124615.1', 'GCF_000894435.1', 'GCF_000870505.1', 'GCF_002816235.1', 'GCF_000864885.1', 'GCF_000870985.1', 'GCF_000868165.1', 'GCF_002816175.1', 'GCF_001504755.1', 'GCF_000896895.1', 'GCF_000862965.1', 'GCF_012271575.1', 'GCF_002816215.1', 'GCF_002816195.1', 'GCF_000896035.1', 'GCF_009858895.2', 'GCF_000879875.1', 'GCF_000853505.1', 'GCF_000896935.1', 'GCF_001725835.1', 'GCF_000880835.1', 'GCF_000875645.1', 'GCF_023122875.1', 'GCF_000919475.1', 'GCF_000901155.1', 'GCF_000883

In [100]:
#filter rows with shared assemblies

spike_filt = spike[spike['Assembly'].isin(shared_nuc_spike_assembly)] 
nuc_filt = nuc[nuc['Assembly'].isin(shared_nuc_spike_assembly)] 

print(spike_filt['Assembly'].count())
print(nuc_filt['Assembly'].count())

60
61


In [101]:
#extract accession ids

spike_acc = spike_filt['Accession']
nuc_acc = nuc_filt['Accession']

print(nuc_acc.head())

0    YP_010037478.1
1    YP_010037564.1
2    YP_010115720.1
3    YP_010799347.1
4    YP_009825061.1
Name: Accession, dtype: object


In [112]:
#extract sequences with accession ids

#nucleocapsid
keep = False
with open("nucleocapsid.fasta") as f_in, open ("nucleocapsid_filtered.fasta", "w") as f_out: 
    for line in f_in:
        if line.startswith(">"):
            acc_id = line[1:].strip().split(" ")[0]
            if acc_id in set(nuc_acc): 
                keep = True
                f_out.write(line)
            else: 
                keep = False
        else: 
            if keep == True: 
                f_out.write(line) 
            

In [113]:
#spike
with open("spike.fasta") as f_in, open ("spike_filtered.fasta", "w") as f_out: 
    for line in f_in:
        if line.startswith(">"):
            acc_id = line[1:].strip().split(" ")[0]
            if acc_id in set(spike_acc): 
                keep = True
                f_out.write(line)
            else: 
                keep = False
        else: 
            if keep == True: 
                f_out.write(line) 


In [14]:
#rewrite headers of aligned fasta files

#nucleocapsid
with open("nucleocapsid_aligned.fasta") as aln_file, open("nucleocapsid_aln_clean.fasta", "w") as clean_file:
    for line in aln_file: 
        if line.startswith(">"):
            acc = line[1:].strip().split(" ")[0]
            gene = line.split("|")[1].split("[")[0].strip().replace(" ", "_")
            org_name = line.strip().split("[")[1].split("]")[0].replace(" ", "_")
            new_head = f">{org_name} | {acc} | {gene}"
            clean_file.write(new_head + "\n") 
        else: 
            clean_file.write(line) 

In [16]:
#repeat for spike

with open("spike_aligned.fasta") as aln_file, open("spike_aln_clean.fasta", "w") as clean_file:
    for line in aln_file: 
        if line.startswith(">"):
            acc = line[1:].strip().split(" ")[0]
            gene = line.split("|")[1].split("[")[0].strip().replace(" ", "_")
            org_name = line.strip().split("[")[1].split("]")[0].replace(" ", "_")
            new_head = f">{org_name} | {acc} | {gene}"
            clean_file.write(new_head + "\n") 
        else: 
            clean_file.write(line)  